# Ponder and Prove the Post Correspondence Problem
## And other undecidability proofs
### Due: Saturday, 13 March 2021, 11:59 pm

## Short Description

The input is a set of *tiles*. Each tile has two ASCII strings written on it, one above the other. For example,

$$\left[\frac{abc}{yz}\right]$$

is a single tile. So, an example of the problem input would be

$$\left\{\left[\frac{yz}{y}\right],\left[\frac{x}{zx}\right],\left[\frac{zz}{z}\right],\left[\frac{ww}{w}\right]\right\}$$

We imagine that we have an infinite supply of each kind of tile in the set.

The output of the problem is "yes" if some tiles from the supply can be arranged in a sequence (with repetitions permitted) so that the strings on the top and bottom of the sequence are identical.

If no such arrangement is possible, the output is "no".

The example above is a positive ("yes") instance, because the following sequence produces the string $yzxzzxyzx$ on the top and bottom:

$$\left[\frac{yz}{y}\right]\left[\frac{x}{zx}\right]\left[\frac{zz}{z}\right]\left[\frac{x}{zx}\right]\left[\frac{yz}{y}\right]\left[\frac{x}{zx}\right]$$


## Longer Description

### Undecidable

**Undecidability** has been called one of the most philosophically important concepts in the theory of computation.

Theoretical computer science identifies specific problems that cannot be solved by any algorithm, because, by their very nature, their solution may never be found. A procedure can be written to search for it, but there's no guarantee it will succeed.

As you know, this problem, known as the Halting Problem (or $A_{TM}$, your "money language"), is the poster child of undecidability.

As we'll see, the Post Correspondence Problem (PCP) is afflicted with the same non-guarantee that the Halting Problem has.


### The Word Problem

The Word Problem (WP) is to find whether two equivalent sentences can be generated from a language.

The WP is similar to the PCP --- and is also easier to grasp.

A.K. Dewdney describes this problem as follows:

Suppose you have a sentence in a given language. Pick words at random until you find a sentence with an equivalent meaning. As an example, consider a sentence below.

The dog ran home →

The canine ran home →

The canine hurried home →

The canine hurried to his house ...

This type of problem is also referred to by Dewdney as a Thue problem, created by Norwegian mathematician Axel Thue. To solve a Thue problem, we use a dictionary to make substitutions until we find an equivalent sentence. It should be noted that the computer cannot determine equivalency; a dictionary must supply mappings of substitutable phrases.

This example seems to show that the algorithm of substituting words works! The problem is that this algorithm may never terminate. Why? Dewdney points out that the substitutions made along the way may "involve intermediate words that are arbitrarily long," thus causing substitutions that may go on forever, as in the following example:

The boy gave the girl a flower →

The boy showed the girl he likes her with a flower →

The boy risked it all, acting in the name of the love, by giving the girl flowers.

Anyone familiar with dating would realize the first sentence and the last one are equivalent. But look how the sentence grows as equivalent meanings are unearthed! We could easily go on and on forever. Because this solution may take forever to return (if at all), this problem is not decidable.

### The Post Correspondence Problem

As with the WP, the PCP is (simply) a specific instance of an undecidable problem. Its creator, Emil L. Post, a Polish-born American mathematician, like Church and Turing, also produced a general model of computation.

The goal of the PCP is to take a set of tiles $\left[\frac{t_1}{b_1}\right], \left[\frac{t_2}{b_2}\right], \cdots, \left[\frac{t_n}{b_n}\right]$, and select them (with repetition) in a certain way. A solution is found when the concatenation of the top portion of the arrangement of tiles is equal to the concatenation of the bottom portion of the tiles.

Ganesh mentions that the details of the proof he just sketches are to be found elsewhere. It is just these "technical" details that make this problem difficult. We'll provide an alternate description that may make it easier to grasp.

We first form a Modified Post Correspondence Problem (MPCP). In this construction, we create tiles based on the transition table for an arbitrary TM, and build the following machines:

D - An instance of the PCP modified so that the first tile is $\left[\frac{t_1}{b_1}\right]$.

M - A Turing Machine constructed by D that accepts a string w.

S - Constructs instance D of the MPCP.

R - Decides the PCP by constructing S.

Note that R is constructed first, then S, then D, then M.

R behaves like $A_{TM}$: it accepts if S accepts, otherwise it rejects (the problem being that "otherwise" includes the case where S doesn't halt!).

Also, $\left[\frac{t_1}{b_1}\right]$ is defined as the initial configuration snapshot (computation history) of M:

$$\left[\frac{\#\hspace{12.5ex}}{\#q_{0}w_{1}w_{2}\ldots w_{n}\#}\right]$$

We construct tiles using computation histories and M's transition function.

The rest of the proof involves describing how these tiles are created and used.

It is important to understand why we even choose to prove that the PCP is undecidable by using computation histories.

As will be shown below, using computation histories allows us to solve the PCP by simulating M on w.

Although there may be other possible algorithms, this one works well --- provided that $A_{TM}$ is decidable (of course, it is not --- but we're playing "what if?").

In order to build R, we construct S such that it builds D according to some rules, as follows:

1. Handle moves to the right. This occurs when we are constructing a partial match where the top tile matches the current head location. Expressed formally, this rule is as follows: For every $a, b \in \Gamma$ and every $q, r \in Q$ where $q \ne q_{reject}$

> if $\delta(q, a) = (r, b, R)$, put $\left[\frac{qa}{br}\right]$ into D.

Note that for this and the next rule, we are simply using the transition function to construct tiles that we can use to "extend the simulation" --- which just means making tiles that create partial matches.

2. Handle moves to the left. This occurs when we find a tape sequence that matches a left movement. For every $a, b, c \in \Gamma$ and every $q, r \in Q$ where $q \ne q_{reject}$,

> if $\delta(q, a) = (r, b, L)$, put $\left[\frac{cqa}{rcb}\right]$ into D.

Note here we may actually create a lot of tiles: one for every $c \in \Gamma$ for every $a, b \in \Gamma$ that matches the transition rule for a left head movement. We inject this $c$ to satisfy the requirement that the head cannot move past the left end of the tape for this modified problem.

3. Handle non-existent moves (tiles for which there is no action). For every $a \in \Gamma$ put $\left[\frac{a}{a}\right]$ into D.

4. For tacking on #'s to delimit the end of each partial match. We simply provide two tiles:

   $\left[\frac{\#}{\#}\right]$ and $\left[\frac{\#}{\sqcup}\right]$.


### Example

Assume M is defined as follows:

In [ ]:
'''
TM
q0: 0; 2, R -> q2
q1: 0; 2, L -> q3
q2: 1; 0, R -> q1
q3: 0; 1, R -> qaccept
'''

1. The first file, as defined for the MPCP, is:

$$\left[\frac{\#\hspace{3ex}}{\#q_{0}\#}\right]$$

2. Tiles created from rule 3:

$$\left[\frac{0}{0}\right], \left[\frac{1}{1}\right], \left[\frac{2}{2}\right], \left[\frac{\sqcup}{\sqcup}\right]$$

3. Tiles created from rule 1:

$$\left[\frac{q_{0}0}{2q_{2}}\right], \left[\frac{q_{2}1}{0q_{1}}\right], \left[\frac{q_{3}0\hspace{3ex}}{1q_{accept}}\right]$$

4. Tiles created from rule 2 using the transition function:

$$\left[\frac{0q_{1}0}{q_{3}02}\right], \left[\frac{1q_{1}0}{q_{3}12}\right], \left[\frac{2q_{1}0}{q_{3}22}\right], \left[\frac{\sqcup q_{1}0}{q_{3}\sqcup 2}\right]$$

5. From rule 4 we have:

$$\left[\frac{\#}{\#}\right]\ \mbox{and}\ \left[\frac{\#}{\sqcup}\right].$$

6. One final rule applies once we have reached an accept state on the bottom. In this case, we can use the following tiles to "pinch off" the top portion. For every $a \in \Gamma$, generate the tile-pair:

$$\left[\frac{aq_{accept}}{q_{accept}}\right], \left[\frac{q_{accept}a}{q_{accept}}\right]$$

Using the generated tiles, we can find the solution to the PCP by selecting partial matches that effectively "simulate $M$ on $w$".

This can be seen in the following sequence of events:

1. From tile set 1,

$$\left[\frac{\#\hspace{7ex}}{\#q_{0}0100\#}\right]$$

2. From tile sets 3, 2, 2, 2, and 5 we select

$$\left[\frac{q_{0}0}{2q_{2}}\right], \left[\frac{1}{1}\right], \left[\frac{0}{0}\right], \left[\frac{0}{0}\right], \left[\frac{\#}{\#}\right]$$

respectively and "smoosh" them together (to better see their alignment):

$$\left[\frac{\#q_{0}0100\#\hspace{7ex}}{\#q_{0}0100\#2q_{2}100\#}\right]$$

3. From tile sets 2, 3, 2, 2, 5 we select

$$\left[\frac{2}{2}\right], \left[\frac{q_{2}1}{0q_{1}}\right], \left[\frac{0}{0}\right], \left[\frac{0}{0}\right], \left[\frac{\#}{\#}\right]$$

respectively to get:

$$\left[\frac{\#q_{0}0100\#2q_{2}100\#\hspace{7ex}}{\#q_{0}0100\#2q_{2}100\#20q_{1}00\#}\right]$$

4. From tile sets 2, 4, 2, 5 we select

$$\left[\frac{2}{2}\right], \left[\frac{q_{3}0}{1q_{accept}}\right], \left[\frac{2}{2}\right], \left[\frac{0}{0}\right], \left[\frac{\#}{\#}\right]$$

respectively to get:

$$\left[\frac{\#q_{0}0100\#2q_{2}100\#20q_{1}00\#2q_{3}020\#\hspace{10ex}}{\#q_{0}0100\#2q_{2}100\#20q_{1}00\#2q_{3}020\#21q_{accept}20\#}\right]$$

5. We can now "pinch off" the top since the bottom has reached an accept state. Add tiles from tile set 6 to the previous sequence:

$$\left[\frac{\ldots \#21q_{accept}20\#2q_{accept}20\#q_{accept}20\#q_{accept}0\#q_{accept}\#\#}{\ldots \hspace{10ex}\#2q_{accept}20\#q_{accept}20\#q_{accept}0\#q_{accept}\#\#}\right]$$

Note that in this "simulation" we are stepping through $M$ on $w$, and by so doing we produce identical strings on the top and bottom, as will be shown.

As we make these substitutions, it should be apparent that the individual tile values we use to form these two strings are different; these differences can be thought of as equivalent words.

We find at this point a "solution" to this particular instance.

The PCP is undecidable in general, however.

To show this, imagine having a TM that doesn't halt. The substitutions, for example, might go in a circle.

The result would be the dreaded **loop of death** that lurks in all undecidable problems.

It should also be noted that a complete solution would need an additional step that involves converting D (an instance of the MPCP) to P (an instance of the PCP.) This involves many more technical details that essentially boil down to this idea: manipulate the input so that it earmarks the first tile $\left[\frac{t_1}{b_1}\right]$ so that it is always selected first. The details are omitted because they're not essential to understanding this reduction.

### WP and PCP

The PCP can be thought of a Turing Reduction of the WP to a Thue Problem. While the problem of making substitutions is approached in the WP with a dictionary, in the PCP we use tiles. These tiles essentially become entries in a dictionary, with the top of the tile matching the value at the current head position. Thus these problems are "the same" problem because one is reducible to the other.

### Conclusion

Part of the problem in understanding the PCP is that it is thick with subtle details that are not immediately apparent in a cursory reading. By actually stepping through the simulation, it is possible to expose these details. Thus, it really is a matter of doing versus seeing. Although several readings may enlighten you, getting your hands dirty has a much better chance of giving you a much deeper understanding!

## Imports

In [1]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import sys

# -- Detect if in Own Install or in Colab
try:
    import google.colab
    OWN_INSTALL = False
except:
    OWN_INSTALL = True
    
if OWN_INSTALL:
  
  sys.path[0:0] = ['../../../../..',  '../../../../../3rdparty',  
                   '../../../..',  '../../../../3rdparty',  
                   '../../..',     '../../../3rdparty', 
                   '../..',        '../../3rdparty',
                   '..',           '../3rdparty',
                   '../Jove',      '../Jove/jove']

else: # In colab
  ! if [ ! -d Jove ]; then git clone https://github.com/ganeshutah/Jove Jove; fi
  sys.path.append('./Jove')
  sys.path.append('./Jove/jove')

# -- common imports --
from jove.DotBashers import *
from jove.Def_md2mc  import *
from jove.Def_NFA    import *
from jove.Def_DFA    import *
from jove.Def_NFA2RE import *
from jove.Def_RE2NFA import *
from jove.AnimateNFA import *
from jove.PcpJupyterNew import *
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

You may use any of these help commands:
help(ResetStNum)
help(NxtStateStr)

You may use any of these help commands:
help(md2mc)
.. and if you want to dig more, then ..
help(default_line_attr)
help(length_ok_input_items)
help(union_line_attr_list_fld)
help(extend_rsltdict)
help(form_delta)
help(get_machine_components)

You may use any of these help commands:
help(mkp_dfa)
help(mk_dfa)
help(totalize_dfa)
help(addtosigma_delta)
help(step_dfa)
help(run_dfa)
help(accepts_dfa)
help(comp_dfa)
help(union_dfa)
help(intersect_dfa)
help(pruneUnreach)
help(iso_dfa)
help(langeq_dfa)
help(same_status)
help(h_langeq_dfa)
help(fixptDist)
help(min_dfa)
help(pairFR)
help(state_combos)
help(sepFinNonFin)
help(bash_eql_classes)
help(listminus)
help(bash_1)
help(mk_rep_eqc)
help(F_of)
help(rep_of_s)
help(q0_of)
help(Delta_of)
help(mk_state_eqc_name)

You may use any of these help commands:
help(mk_nfa)
help(totalize_nfa)
help(step_nfa)
help(run_nfa)
help(ec_step_nfa)
help(Eclosure)
help(Echelp)
help(accept

In [15]:
!../pcb -h

PCPSolver Ver 0.0.3 by Ling Zhao (zhao@cs.ualberta.ca) Nov 16, 2003

pcp  : Solving tool for instances of Post's correspondence problem

Usage: 1. Solve PCP instances in a file:
          pcp -d depth -di inc -r run -i inputfile -o outputfile -ni

       2. Scan a PCP subclass:
          pcp -scan -s size -w width -d depth

       3. Randomly generate difficult solvable instances:
          pcp -ran -s size -w width -d depth -t times -l sol_lim
              -o ofilename

          run     - number of runs
          ni      - no iterative search
          di      - depth increment
          depth   - search depth
          times   - number of instances to be tried
          sol_lim - min optimal solution length (only instances satisfying
                    it will be recorded)



## TODO Exercises from the Book

Please do the following Chapter 15 exercises (comments from Ganesh):

### Exercise 15.2.3.1

Using the PCP solver in Jove, determine the
optimal solution length for the PCP instance
$[(100,1), (0,100), (1,0)].$



Comment: This exercise is a warmup and encourages the use of the tool.

#### Kyle's PCP Code:

In [24]:
pcp_instance = [("100","1"),("0","100"),("1","0")]

transpose = lambda l:[*zip(*l)]

table = transpose(pcp_instance)

def pcp_input_file_maker(table,depth,di):
  size = len(table[0])
  width = max(*map(lambda t:max([*map(len,t)]),table))
  top_row = ' '.join(table[0])
  bottom_row = ' '.join(table[1])
  f = open("pcp.in","w")
  f.write(f"{size} {width} {depth} {di}\n{top_row}\n{bottom_row}\n")
  f.close()

pcp_input_file_maker(table,206,1)

!cat pcp.in
!../pcp -i pcp.in -o pcp.out -di 1 -d 100
!cat pcp.out

3 3 206 1
100 0 1
1 100 0
PCPSolver Ver 0.0.3 by Ling Zhao (zhao@cs.ualberta.ca) Nov 16, 2003

Solve instances in file pcp.in
(solvable => pcp.out, unsolvable => nosol.txt, unsolved => unsol.txt):

Parameters: 
  Clock per second: 1000000
  Cache size      : 8192
  Hash table size : 262144
  Iterative depth : 1
  Maximum depth   : 100


Solve instances:
1
Search to depth: 1
node 1 0
Search to depth: 2
node 1 0
Search to depth: 3
node 2 0
Search to depth: 4
node 3 0
Search to depth: 5
node 4 0
Search to depth: 6
node 5 0
Search to depth: 7
node 5 0
Search to depth: 8
node 9 1
Search to depth: 9
node 9 1
Search to depth: 10
node 11 1
Search to depth: 11
node 13 1
Search to depth: 12
node 18 2
Search to depth: 13
node 21 2
Search to depth: 14
node 23 2
Search to depth: 15
node 27 2
Search to depth: 16
node 32 2
Search to depth: 17
node 34 2
Search to depth: 18
node 37 2
Search to depth: 19
node 40 2
Search to depth: 20
node 49 3
Search to depth: 21
node 53 3
Search to depth: 22
node 58 3


The optimal solution appears to be 75 tiles in length, while any other solution uses more tiles. The PCP code is checking in depth incremements of 1 so we don't skip a depth level where another solution might be hiding.

### Exercise 15.2.3.4

Answer the following questions with respect to 
 the unary PCP instance:

1. Using Jove, solve the following unary PCP instance (i.e., where the alphabet is a singleton, namely $\{0\}$): $[(000,00), (0,0000), (0000,000), (0,0000), (000000,0)].$
2. Argue that the unary PCP problem --- PCP over a singleton alphabet ($|\Sigma|=1$) --- is decidable. Do it in two stages:

> (a) Suppose for all tiles $T_i$ in the unary PCP instance we have $len(T_i[0]) < len(T_i[1])$ or $len(T_i[0]) > len(T_i[1])$. Then what can you say about the unary PCP instance's solutions?

> (b) Suppose the above condition does not hold; that is, there are two distinct tiles $T_j$ and $T_k$ in the instance with $len(T_j[0]) < len(T_j[1])$ and $len(T_k[0]) > len(T_k[1])$. Then what can you say about the unary PCP instance's solutions?

#### Work for 1:

In [29]:
unary_pcp = [("000", "00"), ("0", "0000"), ("0000", "000"), ("0", "0000"), ("000000", "0")]
unary_table = transpose(unary_pcp)
pcp_input_file_maker(unary_table,400,1)

!cat pcp.in
!../pcp -i pcp.in -o pcp.out
!cat pcp.out

5 6 400 1
000 0 0000 0 000000
00 0000 000 0000 0
PCPSolver Ver 0.0.3 by Ling Zhao (zhao@cs.ualberta.ca) Nov 16, 2003

Solve instances in file pcp.in
(solvable => pcp.out, unsolvable => nosol.txt, unsolved => unsol.txt):

Parameters: 
  Clock per second: 1000000
  Cache size      : 8192
  Hash table size : 262144
  Iterative depth : 20
  Maximum depth   : 320


Solve instances:
1
Search to depth: 20
node 150 25

Total nodes searched: 150
1 instances:
  1 solvable.
  0 unsolvable.
  0 unsolved.
time: 0.000676

Total time spent: 0 seconds.
 Instance 1:
5 6 4 160 0.000492
000     0       0000    0       000000  
00      0000    000     0000    0       

Solvable!

 1 1 1 1 1
 1 1 1 1 1
Gcd: 1
Visited node in original direction: 150
Visited node in reverse direction: 150
Choose the original direction:
Find the solution in depth: 4 (depth threshold: 20)
  1  1  2  1
Find the solution in depth: 4 (depth threshold: 4)
  1  1  2  3
Find the solution in depth: 4 (depth threshold: 4)
  1  1  4  1

#### Work for 2:

a. If the condition of $len(T_i[0]) < len(T_i[1])$ or $len(T_i[0]) > len(T_i[1])$ is satisfied for a given PCP set of tiles, then no solution can exist.

b. If the condition is not met, then there must be at least one unique solution to the problem along with infinitely many more by simply concatenating a unique solution multiple times. Multiple unique solutions can also be concatenated together. Kyle notes that unique solutions could be compared to prime numbers while the concatenated solutions could be compared to compound numbers.

### Exercise 15.2.3.5

Here is how we can build a mapping reduction from PCP to CFG grammar ambiguity; please fill in missing steps (if any) and argue that the mapping reduction actually works (achieves its purpose). Let

$$A = w_1,w_2,...,w_n$$

and

$$B=x_1,x_2,...,x_n$$

be two lists of words over a finite alphabet $\Sigma$. Let $a_1,a_2,...,a_n$ be symbols that do not appear in any of the $w_i$ or $x_i$. Let $G$ be a CFG

$$(\{S,S_A,S_B\},\Sigma \cup \{a_1,...,a_n\},P,S),$$

where $P$ contains the productions

$S \rightarrow S_A,$

$S \rightarrow S_B,$

For $1 \le i \le n, S_A \rightarrow w_{i}S_{A}a_{i}$

For $1 \le i \le n, S_A \rightarrow w_{i}a_{i}$

For $1 \le i \le n, S_B \rightarrow x_{i}S_{B}a_{i}$, and

For $1 \le i \le n, S_B \rightarrow x_{i}a_{i}$.

#### The missing link
The current explanation fails to show why this CFG is the result of or the mapping reduction of the PCP problem to the CFG grammar. Mainly, it should be specified that A and B are the tops and bottems of the tiles in a given PCP instance. If A and B are a solution, then the grammar is satisfied. The reason why it works is because of the question and statement below.

Now, argue that $G$ is ambiguous if and only if the PCP instance $(A,B)$ has a solution (thus, we may view the process of going from $(A,B)$ to $G$ as a mapping reduction).

#### The Argument

If the PCP instance has a solution, then the rules of $S_A$ and $S_B$ will result in valid solutions to the grammer with the given inputs of A and B. This means that for that given input, their will be two valid parse trees for this CFG grammar for a single input instance. If A and B aren't a solution to the PCP problem, then only one of S_A or S_B will be the correct "path" if any exist.

### Exercise 15.5.1.3

In 15.5, we described a mapping reduction from $A_{TM}$ to $Halt_{TM}$ by producing a machine $M'$ such that checking for the halting of $M'$ on input $w$ is tantamount to checking whether the original machine $M$ accepts $w$. In the same vein, describe a mapping reduction from $Halt_{TM}$ to $A_{TM}$ by producing a machine $M"$ such that checking whether $M"$ accepts $w$ is tantamount to answering whether $M$ halts on $w$. $\textbf{Hint:}$ Make a copy $M_C$ of $M$, and modify the accept or reject label (figure out which) of $M_C$ suitably. Now, amalgamate both machines $M$ and $M_C$ into one single machine (call it $M_{combined}$) such that essentially both machines get run.

Comment: This exercise is good, as it builds on something already done, and it shows how "label surgery" on a TM is done.

#### Student's note
After some time together, Kyle and I didn't really figure this out in time. Thankfully we have been working together a lot more this week.

### Exercise 15.5.1.5

Define

$CFL_{TM} = \{\langle M \rangle : M $ is a TM whose language is context-free.$\}$

Specify a mapping reduction from $A_{TM}$ to $CFL_{TM}$. Please take ideas from the construction in Figure 15.10.


Comment: This exercise is good and not that hard (resembles Fig 15.10) as it does cause some mental pushups to be exercised.